### Instalando o boto3

In [ ]:
pip install boto3 psycopg2 pandas


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importando o DataSet

In [12]:
import pandas as pd

# Carregue o arquivo .csv
base = pd.read_csv("/content/drive/MyDrive/Andre/ADA/dataset_imputado.csv", sep = ",")
base.head()

,REF_DATE,TARGET,VAR2,IDADE,VAR4,VAR5,VAR8
0,01/06/2017;00:00:00+00:00,0,M,34.137,1.0,RO,4.000000
1,18/08/2017;00:00:00+00:00,0,M,40.447,1.0,PB,5.000000
2,30/06/2017;00:00:00+00:00,0,F,33.515,1.0,RS,4.818291
3,05/08/2017;00:00:00+00:00,1,F,25.797,1.0,BA,5.000000
4,29/07/2017;00:00:00+00:00,0,F,54.074,1.0,RS,2.000000


In [ ]:
import pandas as pd
from datetime import datetime

# Convertendo a coluna para o formato YYYY-MM-DD
base['REF_DATE'] = pd.to_datetime(base['REF_DATE'], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")

base['REF_DATE']

### Conexão com o Banco de Dados

In [7]:
import boto3
import psycopg2
from psycopg2 import sql
import pandas as pd

In [8]:
# Configurações do Banco de Dados RDS
db_config = {
    "host": "chatbotdb.cz0yqgoeikkw.us-east-2.rds.amazonaws.com",
    "database": "postgres",
    "user": "postgres",
    "password": "Erdnal31#",
    "port": 5432
}

In [9]:
# Conexão com o RDS PostgreSQL
def connect_rds():
    try:
        conn = psycopg2.connect(
            host=db_config["host"],
            database=db_config["database"],
            user=db_config["user"],
            password=db_config["password"],
            port=db_config["port"]
        )
        print("Conexão bem-sucedida ao RDS!")
        return conn
    except Exception as e:
        print("Erro ao conectar ao banco de dados:", e)
        return None

In [14]:
# Função para criar a tabela
def create_table(conn):
    query = """
    CREATE TABLE IF NOT EXISTS chatbot_data2 (
        ID SERIAL PRIMARY KEY,
        REF_DATE TIMESTAMP,
        TARGET INT,
        VAR2 VARCHAR(1),
        IDADE FLOAT,
        VAR4 FLOAT,
        VAR5 CHAR(2),
        VAR8 FLOAT
    );
    """
    try:
        with conn.cursor() as cursor:
            cursor.execute(query)
            conn.commit()
            print("Tabela criada com sucesso!")
    except Exception as e:
        print("Erro ao criar tabela:", e)

# Conectar e criar a tabela
conn = connect_rds()
if conn:
    create_table(conn)
    conn.close()

Conexão bem-sucedida ao RDS!
Tabela criada com sucesso!


### Carregar os Dados para o Banco

In [27]:
# Função para carregar dados para o RDS
def load_data_to_rds(conn, df):
    try:
        with conn.cursor() as cursor:
            for _, row in df.iterrows():
                cursor.execute(
                    """
                    INSERT INTO chatbot_data2 (REF_DATE, TARGET, VAR2, IDADE, VAR4, VAR5, VAR8)
                    VALUES (%s, %s, %s, %s, %s, %s, %s)
                    """,
                    (row['REF_DATE'], row['TARGET'], row['VAR2'], row['IDADE'],
                     row['VAR4'], row['VAR5'], row['VAR8'])
                )
            conn.commit()
            print("Dados carregados com sucesso!")
    except Exception as e:
        print("Erro ao carregar dados:", e)

# Carregar apenas as colunas relevantes do CSV
#relevant_columns = ['REF_DATE', 'TARGET', 'VAR2', 'IDADE', 'VAR4', 'VAR5', 'VAR8']
#df_relevant = base[relevant_columns]

# Conectar e carregar os dados
conn = connect_rds()
if conn:
    load_data_to_rds(conn, base)
    conn.close()

Conexão bem-sucedida ao RDS!
Dados carregados com sucesso!


### Testando a Conexão e Consultas

In [10]:
def test_query(conn):
    query = "SELECT VAR5, AVG(IDADE) AS avg_age FROM chatbot_data2 GROUP BY VAR5;"
    with conn.cursor() as cursor:
        cursor.execute(query)
        results = cursor.fetchall()
        for row in results:
            print(row)

conn = connect_rds()
if conn:
    test_query(conn)
    conn.close()

Conexão bem-sucedida ao RDS!
('SC', 41.18209613536117)
('RS', 44.34650745117054)
('DF', 42.551331299481916)
('MG', 42.12051825411429)
('RN', 41.03911226606281)
('SP', 42.323646982033864)
('GO', 41.75788332164354)
('AM', 40.59831755910675)
('PA', 42.98737704187461)
('PE', 40.92114127753574)
('PB', 41.25712258139395)
('AP', 38.92478646667804)
('ES', 42.34845747227685)
('TO', 41.53383922020455)
('MT', 40.28971144866033)
('RR', 39.97231645909703)
('PI', 39.717355057664975)
('PR', 41.86102364265529)
('CE', 41.80285409615007)
('BA', 42.940575343417606)
('AC', 40.10722395381253)
('RJ', 45.52719208760764)
('MA', 41.600078661827034)
('AL', 41.475235134733005)
('RO', 40.44026025542849)
('SE', 41.222062220202154)
('MS', 41.550735829700315)
